In [2]:
import requests
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
import MetaTrader5 as mt5
mt5.initialize()
# Replace following with your MT5 Account Login
account=51434456 # 
password="9UpBvVzc"
server = 'ICMarkets-Demo'

In [4]:
base_url = 'https://stats.bis.org/api/v1'

In [5]:
import json

In [6]:
import requests

# Define the base URL for the BIS Stats API
base_url = 'https://stats.bis.org/api/v1'

# Define the endpoint parameters
flow = 'BIS,WS_EER_D,1.0'  # Example: Version 1.0 of the WS_EER_M domain, maintained by the BIS
key = 'D.N.N.AU'  # Example: Monthly nominal rate in the broad basket for Switzerland
start_period = '2000'  # Example: Start year 2000
end_period = '2023'  # Example: End year 2020
detail = 'full'  # Example: All data and documentation

# Construct the endpoint URL
endpoint_url = f'{base_url}/data/{flow}/{key}/all'

# Define the query parameters
query_params = {
    'startPeriod': start_period,
    'endPeriod': end_period,
    'detail': detail
}

# Make the GET request
response = requests.get(endpoint_url, params=query_params)

# Check for a successful response
if response.status_code == 200:
    # The response data format is XML, so we'll just print the text of the response
    print(response.text)
else:
    print(f'Failed to retrieve data: {response.status_code}')

<?xml version="1.0" encoding="UTF-8"?><message:StructureSpecificData xmlns:ss="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/structurespecific" xmlns:footer="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message/footer" xmlns:ns1="urn:sdmx:org.sdmx.infomodel.datastructure.Dataflow=BIS:WS_EER_D(1.0):ObsLevelDim:TIME_PERIOD" xmlns:message="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message" xmlns:common="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/common" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><message:Header><message:ID>IDREF8b0f5781-80c8-419b-9d80-110d388d5484</message:ID><message:Test>false</message:Test><message:Prepared>2023-10-29T09:44:01Z</message:Prepared><message:Sender id="UNKNOWN"></message:Sender><message:Receiver id="ANONYMOUS"></message:Receiver><message:Structure structureID="BIS_WS_EER_D_1_0" namespace="urn:sdmx:org.sdmx.infomodel.datastructure.Dataflow=BIS:WS_EER_D(1.0):ObsLevelDim:TIME_PERIOD" dimensionAtObservation="TIME_PERIOD"

In [7]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

# Define the base URL for the BIS Stats API
base_url = 'https://stats.bis.org/api/v1'

# Define the endpoint parameters
flow = 'BIS,WS_EER_D,1.0'  # Example: Version 1.0 of the WS_EER_M domain, maintained by the BIS
key = 'D.N.N.AU'  #
start_period = '2000'  # Example: Start year 2000
end_period = '2023'  # Example: End year 2020
detail = 'full'  # Example: All data and documentation

# Construct the endpoint URL
endpoint_url = f'{base_url}/data/{flow}/{key}/all'

# Define the query parameters
query_params = {
    'startPeriod': start_period,
    'endPeriod': end_period,
    'detail': detail
}

# Make the GET request
response = requests.get(endpoint_url, params=query_params)

# Check for a successful response
if response.status_code == 200:
    # Assign the text of the response to xml_data
    xml_data = response.text
    
    # Parse the XML data
    root = ET.fromstring(xml_data)
    
    # Initialize empty lists to store the data
    time_periods = []
    obs_values = []
    
    # Iterate through the XML and extract the desired information
    for obs in root.findall(".//Obs"):
        time_period = obs.get('TIME_PERIOD')
        obs_value = obs.get('OBS_VALUE')
        time_periods.append(time_period)
        obs_values.append(obs_value)
    
    # Create a DataFrame
    df = pd.DataFrame({
        'Time_Period': time_periods,
        'OBS_Value': obs_values
    })
    df = df.dropna()
    # Display the DataFrame
    print(df)
else:
    print(f'Failed to retrieve data: {response.status_code}')

     Time_Period OBS_Value
0     2000-01-01       NaN
1     2000-01-02       NaN
2     2000-01-03     99.49
3     2000-01-04     98.92
4     2000-01-05     99.02
...          ...       ...
8692  2023-10-19    100.62
8693  2023-10-20    100.43
8694  2023-10-21       NaN
8695  2023-10-22       NaN
8696  2023-10-23     100.3

[8697 rows x 2 columns]


In [33]:

df['OBS_Value'] = df['OBS_Value'].replace('NaN', np.nan)
# Drop rows with NaN values
df.dropna(subset=['OBS_Value'], inplace=True)
df['OBS_Value'] = df['OBS_Value'].astype(float)
# df['Time_Period'] = pd.to_datetime(df['Time_Period'])
# df = df.set_index('Time_Period')
df.tail(10)

,OBS_Value
Time_Period,
2023-10-10,101.49
2023-10-11,101.56
2023-10-12,101.34
2023-10-13,100.56
2023-10-16,100.64
2023-10-17,101.09
2023-10-18,101.28
2023-10-19,100.62
2023-10-20,100.43


In [37]:

AUDUSD = get_rates('AUDUSD.a', mt5.TIMEFRAME_D1, 6191)

In [41]:
def get_rates(pair1, timeframe, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, timeframe, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    return pair1[['time','open', 'high', 'low', 'close']].set_index('time')

# AUDUSD = get_rates('AUDUSD.a', mt5.TIMEFRAME_D1, 6191)
AUDUSD = AUDUSD.iloc[:-5]
AUDUSD
combined_df = pd.concat([df, AUDUSD['close']], join = 'outer', axis = 1)
combined_df['shifted_OBS_Value'] = combined_df['OBS_Value'].shift(1)
combined_df = combined_df.dropna()
combined_df

,OBS_Value,close,shifted_OBS_Value
2000-01-04,98.92,0.65490,99.49
2000-01-05,99.02,0.65750,98.92
2000-01-06,99.11,0.65330,99.02
2000-01-07,99.01,0.65510,99.11
2000-01-10,99.48,0.65600,99.01
...,...,...,...
2023-09-26,101.70,0.63968,101.74
2023-09-27,101.37,0.63527,101.70
2023-09-28,101.69,0.64275,101.37
2023-09-29,102.77,0.64352,101.69


In [42]:
from statsmodels.tsa.stattools import grangercausalitytests
grangercausalitytests(combined_df[['shifted_OBS_Value', 'close']], 5)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=263.9391, p=0.0000  , df_denom=6167, df_num=1
ssr based chi2 test:   chi2=264.0675, p=0.0000  , df=1
likelihood ratio test: chi2=258.5729, p=0.0000  , df=1
parameter F test:         F=263.9391, p=0.0000  , df_denom=6167, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1201.9439, p=0.0000  , df_denom=6164, df_num=2
ssr based chi2 test:   chi2=2405.8377, p=0.0000  , df=2
likelihood ratio test: chi2=2031.4228, p=0.0000  , df=2
parameter F test:         F=1201.9439, p=0.0000  , df_denom=6164, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1945.5753, p=0.0000  , df_denom=6161, df_num=3
ssr based chi2 test:   chi2=5843.3575, p=0.0000  , df=3
likelihood ratio test: chi2=4110.8365, p=0.0000  , df=3
parameter F test:         F=1945.5753, p=0.0000  , df_denom=6161, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=1603.6

{1: ({'ssr_ftest': (263.93913219116746, 3.7805205279250026e-58, 6167.0, 1),
   'ssr_chi2test': (264.06752807191555, 2.228014785102933e-59, 1),
   'lrtest': (258.572892769218, 3.512324796214835e-58, 1),
   'params_ftest': (263.9391321912397, 3.7805205277923036e-58, 6167.0, 1.0)},
   array([[0., 1., 0.]])]),
 2: ({'ssr_ftest': (1201.9438995771716, 0.0, 6164.0, 2),
   'ssr_chi2test': (2405.8377405877904, 0.0, 2),
   'lrtest': (2031.422830715026, 0.0, 2),
   'params_ftest': (1201.943899577231, 0.0, 6164.0, 2.0)},
   array([[0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]])]),
 3: ({'ssr_ftest': (1945.5753199903154, 0.0, 6161.0, 3),
   'ssr_chi2test': (5843.3575265542595, 0.0, 3),
   'lrtest': (4110.836467216297, 0.0, 3),
   'params_ftest': (1945.575319990433, 0.0, 6161.0, 3.0)},
   array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0., 0., 1., 0.]])]),
 4: ({'ssr_ftest': (1603.6022777501562, 0.0, 6158.0, 4),
   'ssr_chi2test': (6423.7838

In [111]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

# Define the base URL for the BIS Stats API
base_url = 'https://stats.bis.org/api/v1'

# Define the endpoint parameters
flow = 'BIS,WS_XRU_D, 1'  # Example: Version 1.0 of the WS_EER_M domain, maintained by the BIS
key = 'D.N+R.B.AU'  #
start_period = '2000'  # Example: Start year 2000
end_period = '2023'  # Example: End year 2020
detail = 'full'  # Example: All data and documentation

# Construct the endpoint URL
endpoint_url = f'{base_url}/data/{flow}/{key}/all'

# Define the query parameters
query_params = {
    'startPeriod': start_period,
    'endPeriod': end_period,
    'detail': detail
}

# Make the GET request
response = requests.get(endpoint_url, params=query_params)

# Check for a successful response
if response.status_code == 200:
    # Assign the text of the response to xml_data
    xml_data = response.text
    
    # Parse the XML data
    root = ET.fromstring(xml_data)
    
    # Initialize empty lists to store the data
    time_periods = []
    obs_values = []
    
    # Iterate through the XML and extract the desired information
    for obs in root.findall(".//Obs"):
        time_period = obs.get('TIME_PERIOD')
        obs_value = obs.get('OBS_VALUE')
        time_periods.append(time_period)
        obs_values.append(obs_value)
    
    # Create a DataFrame
    df = pd.DataFrame({
        'Time_Period': time_periods,
        'OBS_Value': obs_values
    })
    df = df.dropna()
    # Display the DataFrame
    print(df)
else:
    print(f'Failed to retrieve data: {response.status_code}')

Failed to retrieve data: 404
